In [21]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize

data = pd.read_csv('./data/CompiledTweets.csv')

data.head()

,Tweet,Sentiment
0,So blessed to live in this beautiful Country,1
1,What a beautiful day it is outside,1
2,I hope I can achieve all of my goals this comi...,1
3,Happy about this great weather,1
4,Feeling optimistic about the new year,1


In [23]:
def clean_tweet(tweet):
    tweet = ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())
    tweet = tweet.strip().lower()
    return ' '.join(word_tokenize(tweet))

# ho = hydro ottawa
ho_x_test = data['Tweet'].map(lambda t: clean_tweet(t))
ho_y_test = data['Sentiment']

In [24]:
import pickle
from keras.models import load_model

# load tokenizer: 
# That's the one we will use to vectorize our data where we want to get the prediction
with open('./tokenizer/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# load rnn model
rnn_model = load_model('./models/rnn-model-01-0.8293.hdf5')

Using TensorFlow backend.


In [25]:
from sklearn.metrics import accuracy_score
from keras.preprocessing.sequence import pad_sequences

ho_test_sequences = tokenizer.texts_to_sequences(ho_x_test)
padded_ho_test_sequences = pad_sequences(ho_test_sequences, maxlen=35)

ho_y_pred = rnn_model.predict(padded_ho_test_sequences, verbose=1, batch_size=2048)

ho_y_pred = pd.DataFrame(ho_y_pred, columns=['prediction'])
ho_y_pred['prediction'] = ho_y_pred['prediction'].map(lambda p: 1 if p >= 0.5 else 0)
print('Accuracy: ', accuracy_score(ho_y_test, ho_y_pred)*100)

443/443 [==============================] - 1s 3ms/step
Accuracy:  88.48758465011286
